In [390]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set the default template to 'plotly_dark' for all Plotly figures
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [391]:
df = pd.read_csv('../data/clean/clustered.csv').set_index('id')
df.head()

,age,gender,income,spending_score,membership_years,purchase_frequency,preferred_category,last_purchase_amount,age_range,income_level,spending_score_category,purchase_frequency_category,membership_score,cluster
id,,,,,,,,,,,,,,
1,38,Female,99342,90,3,24,Groceries,113.53,"(30, 40]",Medium,Premium,Medium,Bronze,1
2,21,Female,78852,60,2,42,Sports,41.93,"(20, 30]",Medium,High,High,Bronze,1
3,60,Female,126573,30,2,28,Clothing,424.36,"(50, 60]",High,Medium,Medium,Bronze,1
4,40,Other,47099,74,9,5,Home & Garden,991.93,"(30, 40]",Low,High,Low,Gold,2
5,65,Female,140621,21,3,25,Electronics,347.08,"(60, 70]",High,Low,Medium,Bronze,1


In [392]:
df['cluster'] = df['cluster'].astype('category')

In [393]:
df.select_dtypes('object').columns

Index(['gender', 'preferred_category', 'age_range', 'income_level',
       'spending_score_category', 'purchase_frequency_category',
       'membership_score'],
      dtype='object')

In [394]:
dimensions = df.select_dtypes('object').columns

for dimension in dimensions:

    data = df.groupby(['cluster', dimension], observed=True).size().reset_index(name='count')
    data['group_total'] = data.groupby('cluster', observed=True)['count'].transform('sum')
    data['pct_of_group_total'] = (data['count'] / data['group_total']) * 100

    x = data['cluster'].astype("object")
    y = data['pct_of_group_total']
    fig = px.bar(data, x=x, y=y, color=dimension)
    fig.show()

In [395]:
data = df.groupby('cluster', observed=True)[['last_purchase_amount', 'purchase_frequency']].sum()

for col in data.columns:
    x = data.index
    y = data[col]
    fig = px.bar(data, x=x, y=y)
    fig.show()

In [396]:
data = df.groupby('cluster', observed=True)[['last_purchase_amount', 'purchase_frequency']].mean()

for col in data.columns:
    x = data.index
    y = data[col]
    fig = px.bar(data, x=x, y=y)
    fig.show()

In [397]:
df.dtypes

age                               int64
gender                           object
income                            int64
spending_score                    int64
membership_years                  int64
purchase_frequency                int64
preferred_category               object
last_purchase_amount            float64
age_range                        object
income_level                     object
spending_score_category          object
purchase_frequency_category      object
membership_score                 object
cluster                        category
dtype: object

In [398]:
df_reset = df.reset_index()
dimension = 'income_level'
metric = 'spending_score'

data = df_reset.groupby(['cluster', dimension], observed=True)[[metric, 'id']].agg({metric: 'mean', 'id': 'count'}).round(0).reset_index()
fig = px.scatter(data, x=data[dimension], y=data[metric], size='id', color='cluster', size_max=60)
fig.show()

In [399]:


# Reset the index
df_reset = df.reset_index()

# Dimensions and axes
dims = ['age_range', 'gender', 'income_level', 'spending_score_category', 'membership_score', 'preferred_category']
x_axis = 'last_purchase_amount'
y_axis = 'purchase_frequency'

# Create subplots
fig = make_subplots(
    rows=len(dims),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05
)

# Loop through dimensions and add traces
for i, dim in enumerate(dims):
    # Group by cluster and the current dimension, calculate means and counts
    dim_df = df_reset.groupby(['cluster', dim], observed=True)[[x_axis, y_axis]].mean().reset_index()
    dim_df['id'] = df_reset.groupby(['cluster', dim], observed=True).size().values  # Count occurrences

    
    # Add scatter plot trace
    fig.add_trace(go.Scatter(
        x=dim_df[x_axis],
        y=dim_df[y_axis],
        mode='markers',
        marker=dict(
            size=dim_df['id'], 
            sizemode='area',  # Size circles proportionally to count
            sizeref=2. * max(dim_df['id']) / 100,  # Adjust marker size
            sizemin=5  # Minimum size of a marker
        ),
        text=dim_df[dim],  # Optional: Add hover text
        name=f"Cluster: {dim}"
    ),
    row=(i + 1),
    col=1)

# Adjust layout
fig.update_layout(
    height=300 * len(dims),  # Dynamic height adjustment
    title="Analysis of Clusters by Dimensions",
    xaxis_title=x_axis,
    yaxis_title=y_axis,
    showlegend=False  # Set to True if you want a legend
)

# Show the figure
fig.show()


In [400]:
df_reset = df.reset_index()
dims = ['age_range', 'gender', 'income_level', 'spending_score_category', 'membership_score', 'preferred_category']
x_axis = 'last_purchase_amount'
y_axis = 'purchase_frequency'
g1 = 'cluster'
for dim in dims:
    data = df_reset.groupby([g1, dim], observed=True)[[x_axis, y_axis, 'id']].agg({x_axis: 'mean', y_axis: 'mean', 'id': 'count'}).round(0).reset_index()
    fig = px.scatter(data, x=data[x_axis], y=data[y_axis], size='id', color=g1, size_max=60, hover_data=[dim])
    fig.show()

In [401]:
fig = px.sunburst(df, path=['cluster', 'membership_score'])
fig.show()

/Users/anthonydillon/Projects/customer_segmentation/cust-seg-venv/lib/python3.9/site-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/anthonydillon/Projects/customer_segmentation/cust-seg-venv/lib/python3.9/site-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [402]:
data = df.groupby(['cluster', 'preferred_category'], observed=True)['spending_score'].mean().reset_index().round(0).sort_values(by=['cluster', 'spending_score'], ascending=[True, True])

clusters = data['cluster'].unique()

fig = make_subplots(
    rows=len(clusters),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05
)

fig.update_layout(margin=dict(l=250))

for i, cluster in enumerate(clusters):

    cat_df = data[data['cluster'] == cluster]

    fig.add_trace(go.Bar(
        x=cat_df['spending_score'],
        y=cat_df['preferred_category'],
        orientation='h',
        name=f"Cluster: {cluster}",
        text=cat_df['spending_score'],
        textfont=dict(color='white')
    ),
    row=i+1,
    col=1)

    fig.add_annotation(
        xref='paper',
        yref='y' + str(i + 1),
        xanchor='right',
        x=-0.12,
        y=cat_df['preferred_category'].iloc[len(cat_df) // 2],
        text=f"Cluster: {cluster}",
        showarrow=False,
        font=dict(size=12)
    )
    
    fig.update_yaxes(
        showline=True,
        linecolor='lightgrey',
        linewidth=1,
        ticklabelposition='outside',
        ticklen=7,
        tickcolor='white',
        row=i+1,
        col=1
    )


fig.show()

In [403]:
data = df.groupby(['cluster', 'preferred_category'], observed=True).size().reset_index(name='count').sort_values(by=['cluster', 'count'], ascending=[True, True])
data['group_total'] = data.groupby('cluster', observed=True)['count'].transform('sum')
data['pct_of_group_total'] = ((data['count'] / data['group_total']) * 100).round(0)
display(data)

clusters = data['cluster'].unique()

fig = make_subplots(
    rows=len(clusters),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05
)

fig.update_layout(margin=dict(l=250))

for i, cluster in enumerate(clusters):

    cat_df = data[data['cluster'] == cluster]

    fig.add_trace(go.Bar(
        x=cat_df['pct_of_group_total'],
        y=cat_df['preferred_category'],
        orientation='h',
        name=f"Cluster: {cluster}",
        text=cat_df['pct_of_group_total'],
        textfont=dict(color='white')
    ),
    row=i+1,
    col=1)

    fig.add_annotation(
        xref='paper',
        yref='y' + str(i + 1),
        xanchor='right',
        x=-0.12,
        y=cat_df['preferred_category'].iloc[len(cat_df) // 2],
        text=f"Cluster: {cluster}",
        showarrow=False,
        font=dict(size=12)
    )
    
    fig.update_yaxes(
        showline=True,
        linecolor='lightgrey',
        linewidth=1,
        ticklabelposition='outside',
        ticklen=7,
        tickcolor='white',
        row=i+1,
        col=1
    )


fig.show()

,cluster,preferred_category,count,group_total,pct_of_group_total
0,0,Clothing,70,431,16.0
2,0,Groceries,82,431,19.0
4,0,Sports,91,431,21.0
1,0,Electronics,93,431,22.0
3,0,Home & Garden,95,431,22.0
8,1,Home & Garden,58,318,18.0
5,1,Clothing,64,318,20.0
7,1,Groceries,64,318,20.0
9,1,Sports,64,318,20.0
6,1,Electronics,68,318,21.0
